In [2]:
!pip install camb

  Using cached camb-1.5.4.tar.gz (1.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Created wheel for camb: filename=camb-1.5.4-py3-none-any.whl size=1988431 sha256=c211735372ee4128c9824c81677d4b83d3ceafc0aea9aef7a116491726c18335
  Stored in directory: /Users/henrybae/Library/Caches/pip/wheels/e6/bd/75/17f7c98e7f88328a18c8cec889e391f5eb5be8c7258a9b9783
Successfully built camb


In [9]:
import camb
import numpy as np
import os
import likelihood

def calculate_power_spectra(params, lensed=True):
    # Set up a new set of parameters for CAMB
    pars = camb.set_params(**params)
    # Calculate results for these parameters
    results = camb.get_results(pars)
    # Get dictionary of CAMB power spectra
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')
    # Choose the appropriate power spectrum
    if lensed:
        cl = powers['total']
    else:
        cl = powers['unlensed_scalar']
    # Cut out the first 2 elements of the power spectra, which are zero
    cl_cut = cl[2:]
    # Get the TT, TE, and EE power spectra
    cl_tt = cl_cut[:, 0]
    cl_te = cl_cut[:, 3]
    cl_ee = cl_cut[:, 1]
    return cl_tt, cl_te, cl_ee

params = {
    # Parameters that are part of the proposal distribution:
    'ombh2': 0.025,                # Baryon density parameter
    'omch2': 0.122,                # Cold dark matter density parameter
    'cosmomc_theta': 1.04109/100,       # Theta_MC - need to be transformed
    'tau': 0.06,                   # Optical depth to reionization
    'ns': 0.965,                   # Scalar spectral index
    'As': 2.0e-9,# the scalar amplitude, need to be transformed for proposal distribution

    # Fixed parameters:
    #'H0': 69.5,                    # Hubble constant in km/s/Mpc (Fixed)
    'mnu': 0.06,                   # Sum of neutrino masses in eV (Fixed)
    'omk': 0,                      # Curvature density parameter, flat universe (Fixed)
    'halofit_version': 'mead',     # Non-linear matter power spectrum model version (Fixed)
    'lmax': 2800                   # Maximum multipole number for calculations (Fixed)
}

likelihood_function = likelihood.PlanckLitePy(spectra='TT')

cl_tt, cl_te, cl_ee = calculate_power_spectra(params, lensed=True)
print(cl_tt)
print(cl_te)
print(cl_ee)



[982.32562024 927.57538209 875.41964977 ...  36.22063115  36.11447954
  36.00921749]
[ 2.75474425  3.09469103  2.90353789 ... -1.26516941 -1.27129668
 -1.27752765]
[0.03586869 0.04609061 0.04003475 ... 1.56565523 1.56519372 1.56468134]


In [4]:
def process_params_and_calculate_spectra(array):
    array = array.copy()  # Create a copy of the array
    # Modify the third and sixth elements of the array
    array[2] /= 100
    array[5] = np.exp(array[5]) / 1e10

    # Set the parameters for the proposal distribution
    params = {
        'ombh2': array[0],
        'omch2': array[1],
        'cosmomc_theta': array[2],
        'tau': array[3],
        'ns': array[4],
        'As': array[5],
 
        
        'mnu': 0.06,
        'omk': 0,
        'halofit_version': 'mead',
        'lmax': 2800
    }

    # Call the calculate_power_spectra function and return the result
    return calculate_power_spectra(params)


def likelihoodfunc(array):
    cl_tt, cl_te, cl_ee = process_params_and_calculate_spectra(array)
    return likelihood_function.loglike(cl_tt, cl_te, cl_ee)

In [5]:
def proposal_distribution(current_state):
    step_sizes = np.array([
        np.sqrt(0.0001**2),  # ombh2
        np.sqrt(0.001**2),   # omch2
        np.sqrt(0.0004**2),  # theta_MC_100
        np.sqrt(0.006**2),   # tau
        np.sqrt(0.004**2),   # ns
        np.sqrt(0.001**2)    # log(10^10 As)
    ])
    proposal_step = np.random.normal(0, step_sizes)
    return current_state + proposal_step



def proposal_distribution_2(current_state):
    step_sizes = np.array([
        np.sqrt(0.001**2),  # ombh2
        np.sqrt(0.001**2),   # omch2
        np.sqrt(0.0004**2),  # theta_MC_100
        np.sqrt(0.006**2),   # tau
        np.sqrt(0.004**2),   # ns
        np.sqrt(0.01**2)    # log(10^10 As)
    ])
    proposal_step = np.random.normal(0, step_sizes)
    return current_state + proposal_step

def proposal_distribution_3(current_state):
    step_sizes = np.array([
        0.0001,  # ombh2
        0.0005,  # omch2
        0.0002,  # theta_MC_100
        0.003,   # tau
        0.002,   # ns
        0.001    # log(10^10 As)
    ])
    proposal_step = np.random.normal(0, step_sizes)
    return current_state + proposal_step

In [7]:
!pip install lenstools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.6/259.6 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for lenstools: filename=lenstools-1.2-cp39-cp39-macosx_14_0_arm64.whl size=314462 sha256=9979540fe3153a8a46c339590cf9bdfd80b7958b8a5961a518d57b6b11e92f82
  Stored in directory: /Users/henrybae/Library/Caches/pip/wheels/01/fd/7a/bcad0ec1952249ea1549ff746c73e14bd9f086a63dc58ccaa4
  Created wheel for emcee: filename=emcee-2.2.1-py3-none-any.whl size=29592 sha256=f745a076abee92ff9ce27e08ad19dc52bce27a8904c960a2641df7aa256afa81
  Stored in directory: /Users/henrybae/Library/Caches/pip/wheels/a2/78/08/02ba0bb42f9b3ce7538ba285b8027b261cffa4414d55c5267d
Successfully built lenstools emcee


In [2]:
import tensorflow as tf

In [6]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

va = 1.0
priors = [
    tfd.Normal(loc=0.0224, scale=tf.sqrt(va)*0.0001),  # ombh2
    tfd.Normal(loc=0.12, scale=tf.sqrt(va)*0.001),  # omch2
    tfd.Normal(loc=1.04109, scale=tf.sqrt(va)*0.0004),  # theta_MC_100
    tfd.Normal(loc=0.055, scale=tf.sqrt(va)*0.006),  # tau
    tfd.Normal(loc=0.965, scale=tf.sqrt(va)*0.004),  # ns
    tfd.Normal(loc=3.05, scale=tf.sqrt(va)*0.001)  # log(10^10 As)
]

2024-04-17 23:23:28.726091: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-04-17 23:23:28.726119: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-04-17 23:23:28.726124: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-04-17 23:23:28.726283: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-17 23:23:28.726299: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
def proposal_distribution_3(current_state):
    step_sizes = np.array([
        0.0001,  # ombh2
        0.0005,  # omch2
        0.0002,  # theta_MC_100
        0.003,   # tau
        0.002,   # ns
        0.001    # log(10^10 As)
    ])
    current_state = np.stack(current_state, axis=0)  # Convert list of tensors to a tensor
    num_chains = current_state.shape[1]
    proposal_step = np.random.normal(0, step_sizes[:, None], size=(6, num_chains))
    return current_state + proposal_step

In [23]:
class MCMC_HMC:
    def __init__(self, likelihood_function, priors, num_samples, num_chains, step_size=0.1, num_burnin_steps=1000):
        self.likelihood_function = likelihood_function
        self.priors = priors
        self.num_samples = num_samples
        self.num_chains = num_chains
        self.step_size = step_size
        self.num_burnin_steps = num_burnin_steps

        # Define the HMC kernel with adaptive step size
        self.kernel = tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.HamiltonianMonteCarlo(
                target_log_prob_fn=self.joint_log_prob,
                num_leapfrog_steps=3,
                step_size=self.step_size,
                state_gradients_are_stopped=True),
            num_adaptation_steps=int(num_burnin_steps * 0.8))

    def joint_log_prob(self, *params):
        """Calculate the joint log probability of model and priors."""
        params = tf.concat(params, axis=0)  # Concatenate the parameters into a single array
        prior_log_prob = sum(prior.log_prob(param) for prior, param in zip(self.priors, params))
        likelihood_log_prob = self.likelihood_function(*params)
        return prior_log_prob + likelihood_log_prob

    def run_chain(self):
        # Initial state by sampling from the proposal distribution
        init_state = proposal_distribution_3([prior.sample([self.num_chains]) for prior in self.priors])

        # Convert list of states into a tuple of states needed for TFP
        init_state = tuple(init_state)

        # Running the MCMC chain
        samples, is_accepted = tfp.mcmc.sample_chain(
            num_results=self.num_samples,
            num_burnin_steps=self.num_burnin_steps,
            current_state=init_state,
            kernel=self.kernel,
            trace_fn=lambda _, pkr: pkr.inner_results.is_accepted)

        return samples, tf.reduce_mean(tf.cast(is_accepted, dtype=tf.float32))

    def sample(self):
        samples, acceptance_rate = self.run_chain()
        print(f'Acceptance rate: {acceptance_rate.numpy()}')
        return samples

In [24]:
mcmc = MCMC_HMC(likelihoodfunc, priors, num_samples=10, num_chains=4, step_size=0.1, num_burnin_steps=10)
samples = mcmc.sample()

TypeError: `x` and `y` must have the same dtype, got tf.float64 != tf.float32.

In [8]:
!pip install pandas
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.5 MB/s eta 0:00:0000:0100:01m


In [10]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 17.9 MB/s eta 0:00:0000:0100:01


In [12]:
!pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 5.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 17.2 MB/s eta 0:00:00
